In [21]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import json

In [22]:
url = "http://fund.eastmoney.com/js/fundcode_search.js" 
response = requests.get(url)
soup = BeautifulSoup(response.content,'lxml')

string = soup.get_text()
pattern = r'\D(\d{6})\D'
fund_code_list = re.findall(pattern,string)  ### 获取市场上所有基金代码

def fund_code(i):
    return fund_code_list[i]

###对单只基金的净值数据进行抓取
def fund_data(i):
    url_i = 'http://fund.eastmoney.com/'+ fund_code(i)+'.html'   
    response_i = requests.get(url_i)
    soup_i = BeautifulSoup(response_i.content,'lxml')

    today = soup_i.find('dl',class_='dataItem01')
    yesterday = soup_i.find('dl',class_='dataItem02')
    summary = soup_i.find('dl',class_='dataItem03')
    
    name = soup_i.find_all('div',class_='SitePath')
    fund_info = soup_i.find_all('div',class_='infoOfFund')
    
    if today == None:
        fund_name = 0
        fund_type = 0
        fund_manager = 0
        zdf_1year = 0
        zdf_1month = 0
        dwjz_today = 0
        zdf_today = 0
        zdf_3year =0
        zdf_3month = 0
        zdf_yesterday = 0
        dwjz_yesterday = 0
        zdf_sincestart =0
        zdf_6month = 0
        ljjz = 0
    else:
        try:
            fund_name = name[0].find_all('a')[2].get_text()
            fund_type = fund_info[0].find_all('a')[0].get_text()
            fund_manager = fund_info[0].find_all('a')[2].get_text()

            data_today = today.find_all('span')
            zdf_1year = data_today[-1].get_text()
            zdf_1month = data_today[-3].get_text()
            dwjz_today = data_today[-7].get_text()
            zdf_today = data_today[-5].get_text()

            data_yesterday = yesterday.find_all('span')
            zdf_3year = data_yesterday[-1].get_text()
            zdf_3month = data_yesterday[-3].get_text()
            zdf_yesterday = data_yesterday[-5].get_text()
            dwjz_yesterday = data_yesterday[-6].get_text()

            data_summary = summary.find_all('span')
            zdf_sincestart = data_summary[-1].get_text()
            zdf_6month = data_summary[-3].get_text()
            ljjz = data_summary[-5].get_text()  
            
        except:   ###货币型基金数据结构 与 股票型、债券型等基金不一致，先暂时剔除
            fund_name = 0
            fund_type = 0
            fund_manager = 0
            zdf_1year = 0
            zdf_1month = 0
            dwjz_today = 0
            zdf_today = 0
            zdf_3year =0
            zdf_3month = 0
            zdf_yesterday = 0
            dwjz_yesterday = 0
            zdf_sincestart =0
            zdf_6month = 0
            ljjz = 0
        
    fund_info = [fund_code(i),fund_name,fund_type,fund_manager,dwjz_today,dwjz_yesterday,ljjz,zdf_today,zdf_1month,zdf_3month,zdf_6month,zdf_1year,zdf_3year,zdf_sincestart] 
    
    return fund_info

In [23]:
result = [fund_data(i) for i in range(len(fund_code_list))]  ### 对市场上所有基金进行遍历，获取每天净值数据

In [68]:
df = pd.DataFrame(result)
df.columns=['基金代码','基金名称','基金类型','基金经理','今日净值','昨日净值','累计净值','今日涨跌幅','过去一月涨跌幅','过去三月涨跌幅',\
            '过去六月涨跌幅','过去一年涨跌幅','过去三年涨跌幅','成立以来涨跌幅',]
df.index = np.arange(1,len(df)+1,1)
df_new = df[df['基金名称']!=0]
df_new.index = np.arange(1,len(df_new)+1,1)
#df_new.to_csv('C:\\Users\\lujiani\\Desktop\\东方证券\\珺容\\基金数据库\\20200610.csv',index=False, encoding='utf_8_sig') ###生成excel


In [69]:
df_new

,基金代码,基金名称,基金类型,基金经理,今日净值,昨日净值,累计净值,今日涨跌幅,过去一月涨跌幅,过去三月涨跌幅,过去六月涨跌幅,过去一年涨跌幅,过去三年涨跌幅,成立以来涨跌幅
1,000001,华夏成长混合,混合型,董阳阳,1.2089,1.2050,3.6660,+0.33%,3.61%,0.75%,16.65%,18.90%,25.66%,591.02%
2,000003,中海可转债债券A,债券型,彭海平,0.7625,0.7660,0.9760,-0.46%,-4.01%,-8.70%,3.79%,13.48%,0.26%,-9.94%
3,000004,中海可转债债券C,债券型,彭海平,0.7615,0.7650,0.9750,-0.46%,-4.02%,-8.71%,3.52%,13.33%,-0.39%,-10.06%
4,000005,嘉实增强信用定期债券,定开债券,刘宁,1.0405,1.0400,1.3820,+0.04%,-1.42%,0.69%,3.96%,7.94%,21.49%,44.58%
5,000006,西部利得量化成长混合,混合型,盛丰衍,1.4667,1.4656,1.4656,+0.08%,5.78%,3.95%,30.72%,52.86%,--,46.56%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9249,970002,天泽六个月定开债C,定开债券,吴一纯,0.9998,0.9998,0.9998,0.00%,--,--,--,--,--,-0.02%
9250,970003,安信瑞鸿中短债A,债券型,杨坚丽,1.0233,1.0233,1.4747,0.00%,--,--,--,--,--,0.03%
9251,970004,安信瑞鸿中短债B,债券型,杨坚丽,1.0234,1.0234,1.0234,0.00%,--,--,--,--,--,0.03%
9252,970005,安信瑞鸿中短债C,债券型,杨坚丽,1.0233,1.0233,1.0233,0.00%,--,--,--,--,--,0.02%


## 债券型基金细分类

### 长期纯债

In [4]:
def CQCZ_fundcode():
    total_list = np.array([])
    i=1
    url_i = 'http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=13&letter=&gsid=&text=&sort=zdf,\
               desc&page=%s,200&feature=041|&dt=1591237233863&atfc=&onlySale=0' % i
    response_i = requests.get(url_i)
    soup_i = BeautifulSoup(response_i.content,'lxml')
    string_i = soup_i.get_text()

    pattern_totalpages = r'pages:\s*"\d{1,9}'  #"speed"\s*:\s*"(\d+)"
    match1 = re.findall(pattern_totalpages,string_i)
    pattern_pagenum = r'\d{1,9}'
    total_pages = int(re.findall(pattern_pagenum,str(match1))[0])
    
    while i<=total_pages:
        pattern = r'\D(\d{6})\D'
        fund_code_list = re.findall(pattern,string_i)
        total_list = np.append(total_list,fund_code_list)
        i=i+1
        
        url_i = 'http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=13&letter=&gsid=&text=&sort=zdf,\
        desc&page=%s,200&feature=041|&dt=1591250844954&atfc=&onlySale=0'  % i
        response_i = requests.get(url_i)
        soup_i = BeautifulSoup(response_i.content,'lxml')
        string_i = soup_i.get_text()
    
    return total_list   

In [26]:
CQCZ_list = list(CQCZ_fundcode())

## 短期纯债

In [ ]:
# http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=13&letter=&gsid=&text=&sort=zdf,\
#     desc&page=1,200&feature=042|&dt=1591251297443&atfc=&onlySale=0

In [6]:
def DQCZ_fundcode():
    total_list = np.array([])
    i=1
    url_i = 'http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=13&letter=&gsid=&text=&sort=zdf,\
    desc&page=%s,200&feature=042|&dt=1591251297443&atfc=&onlySale=0' % i
    response_i = requests.get(url_i)
    soup_i = BeautifulSoup(response_i.content,'lxml')
    string_i = soup_i.get_text()

    pattern_totalpages = r'pages:\s*"\d{1,9}'  #"speed"\s*:\s*"(\d+)"
    match1 = re.findall(pattern_totalpages,string_i)
    pattern_pagenum = r'\d{1,9}'
    total_pages = int(re.findall(pattern_pagenum,str(match1))[0])
    
    while i<=total_pages:
        pattern = r'\D(\d{6})\D'
        fund_code_list = re.findall(pattern,string_i)
        total_list = np.append(total_list,fund_code_list)
        i=i+1
        
        url_i = 'http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=13&letter=&gsid=&text=&sort=zdf,\
    desc&page=%s,200&feature=042|&dt=1591251297443&atfc=&onlySale=0' % i
        response_i = requests.get(url_i)
        soup_i = BeautifulSoup(response_i.content,'lxml')
        string_i = soup_i.get_text()
    
    return total_list
        

In [27]:
DQCZ_list = list(DQCZ_fundcode())

## 混合债基

In [ ]:
#http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=13&letter=&gsid=&text=&sort=zdf,\
#desc&page=1,200&feature=043|&dt=1591251914495&atfc=&onlySale=0

In [10]:
def HHZJ_fundcode():
    total_list = np.array([])
    i=1
    url_i = 'http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=13&letter=&gsid=&text=&sort=zdf,\
desc&page=%s,200&feature=043|&dt=1591251914495&atfc=&onlySale=0' % i
    response_i = requests.get(url_i)
    soup_i = BeautifulSoup(response_i.content,'lxml')
    string_i = soup_i.get_text()

    pattern_totalpages = r'pages:\s*"\d{1,9}'  
    match1 = re.findall(pattern_totalpages,string_i)
    pattern_pagenum = r'\d{1,9}'
    total_pages = int(re.findall(pattern_pagenum,str(match1))[0])
    
    while i<=total_pages:
        pattern = r'\D(\d{6})\D'
        fund_code_list = re.findall(pattern,string_i)
        total_list = np.append(total_list,fund_code_list)
        i=i+1
        
        url_i = 'http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=13&letter=&gsid=&text=&sort=zdf,\
desc&page=%s,200&feature=043|&dt=1591251914495&atfc=&onlySale=0' % i
        response_i = requests.get(url_i)
        soup_i = BeautifulSoup(response_i.content,'lxml')
        string_i = soup_i.get_text()
    
    return total_list

In [28]:
HHZJ_list = list(HHZJ_fundcode())

## 定期开放债券

In [ ]:
#http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=13&letter=&gsid=&text=&sort=zdf,\
            #desc&page=1,200&feature=008|&dt=1591252127082&atfc=&onlySale=0

In [13]:
def DQKFZJ_fundcode():
    total_list = np.array([])
    i=1
    url_i = 'http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=13&letter=&gsid=&text=&sort=zdf,\
            desc&page=%s,200&feature=008|&dt=1591252127082&atfc=&onlySale=0' % i
    response_i = requests.get(url_i)
    soup_i = BeautifulSoup(response_i.content,'lxml')
    string_i = soup_i.get_text()

    pattern_totalpages = r'pages:\s*"\d{1,9}'  
    match1 = re.findall(pattern_totalpages,string_i)
    pattern_pagenum = r'\d{1,9}'
    total_pages = int(re.findall(pattern_pagenum,str(match1))[0])
    
    while i<=total_pages:
        pattern = r'\D(\d{6})\D'
        fund_code_list = re.findall(pattern,string_i)
        total_list = np.append(total_list,fund_code_list)
        i=i+1
        
        url_i = 'http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=13&letter=&gsid=&text=&sort=zdf,\
            desc&page=%s,200&feature=008|&dt=1591252127082&atfc=&onlySale=0' % i
        response_i = requests.get(url_i)
        soup_i = BeautifulSoup(response_i.content,'lxml')
        string_i = soup_i.get_text()
    
    return total_list

In [29]:
DQKFZQ_list = list(DQKFZJ_fundcode())

## 可转债

In [ ]:
#http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=13&letter=&gsid=&text=&sort=zdf,desc&page=1,\
#200&feature=045|&dt=1591252269353&atfc=&onlySale=0

In [15]:
def KZJ_fundcode():
    total_list = np.array([])
    i=1
    url_i = 'http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=13&letter=&gsid=&text=&sort=zdf,\
    desc&page=%s,200&feature=045|&dt=1591252269353&atfc=&onlySale=0' % i
    response_i = requests.get(url_i)
    soup_i = BeautifulSoup(response_i.content,'lxml')
    string_i = soup_i.get_text()

    pattern_totalpages = r'pages:\s*"\d{1,9}'  
    match1 = re.findall(pattern_totalpages,string_i)
    pattern_pagenum = r'\d{1,9}'
    total_pages = int(re.findall(pattern_pagenum,str(match1))[0])
    
    while i<=total_pages:
        pattern = r'\D(\d{6})\D'
        fund_code_list = re.findall(pattern,string_i)
        total_list = np.append(total_list,fund_code_list)
        i=i+1
        
        url_i = 'http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=13&letter=&gsid=&text=&sort=zdf,\
    desc&page=%s,200&feature=045|&dt=1591252269353&atfc=&onlySale=0' % i
        response_i = requests.get(url_i)
        soup_i = BeautifulSoup(response_i.content,'lxml')
        string_i = soup_i.get_text()
    
    return total_list

In [30]:
KZZ_list = list(KZJ_fundcode())

## 被动指数型

In [ ]:
# http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=5&letter=&gsid=&text=&sort=zdf,\
#desc&page=1,9999&feature=|051&dt=1591252580869&atfc=&onlySale=0

In [17]:
def BDZSX_fundcode():
    total_list = np.array([])
    i=1
    url_i = 'http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=5&letter=&gsid=&text=&sort=zdf,\
desc&page=%s,9999&feature=|051&dt=1591252580869&atfc=&onlySale=0' % i
    response_i = requests.get(url_i)
    soup_i = BeautifulSoup(response_i.content,'lxml')
    string_i = soup_i.get_text()

    pattern_totalpages = r'pages:\s*"\d{1,9}'  
    match1 = re.findall(pattern_totalpages,string_i)
    pattern_pagenum = r'\d{1,9}'
    total_pages = int(re.findall(pattern_pagenum,str(match1))[0])
    
    while i<=total_pages:
        pattern = r'\D(\d{6})\D'
        fund_code_list = re.findall(pattern,string_i)
        total_list = np.append(total_list,fund_code_list)
        i=i+1
        
        url_i = 'http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=5&letter=&gsid=&text=&sort=zdf,\
desc&page=%s,9999&feature=|051&dt=1591252580869&atfc=&onlySale=0' % i
        response_i = requests.get(url_i)
        soup_i = BeautifulSoup(response_i.content,'lxml')
        string_i = soup_i.get_text()
    
    return total_list

In [31]:
BDZSX_list = list(BDZSX_fundcode())

## 增强指数型


In [ ]:
#http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=5&letter=&gsid=&text=&sort=zdf,\
#desc&page=1,9999&feature=|052&dt=1591252828202&atfc=&onlySale=0

In [19]:
def ZQZSX_fundcode():
    total_list = np.array([])
    i=1
    url_i = 'http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=5&letter=&gsid=&text=&sort=zdf,\
desc&page=%s,9999&feature=|052&dt=1591252828202&atfc=&onlySale=0' % i
    response_i = requests.get(url_i)
    soup_i = BeautifulSoup(response_i.content,'lxml')
    string_i = soup_i.get_text()

    pattern_totalpages = r'pages:\s*"\d{1,9}'  
    match1 = re.findall(pattern_totalpages,string_i)
    pattern_pagenum = r'\d{1,9}'
    total_pages = int(re.findall(pattern_pagenum,str(match1))[0])
    
    while i<=total_pages:
        pattern = r'\D(\d{6})\D'
        fund_code_list = re.findall(pattern,string_i)
        total_list = np.append(total_list,fund_code_list)
        i=i+1
        
        url_i = 'http://fund.eastmoney.com/Data/Fund_JJJZ_Data.aspx?t=1&lx=5&letter=&gsid=&text=&sort=zdf,\
desc&page=%s,9999&feature=|052&dt=1591252828202&atfc=&onlySale=0' % i
        response_i = requests.get(url_i)
        soup_i = BeautifulSoup(response_i.content,'lxml')
        string_i = soup_i.get_text()
    
    return total_list

In [32]:
ZQZSX_list = list(ZQZSX_fundcode())

## 子分类细分

In [70]:
#def categ_fix(i):
df_new_copy = df_new

for i in list(df_new_copy['基金代码']):
    
    if i in CQCZ_list:
        location = int(np.where(df_new_copy['基金代码']== i )[0])+1
        df_new_copy.loc[location,'基金类型']='长期纯债'
        
    if i in DQCZ_list:
        location = int(np.where(df_new_copy['基金代码']== i )[0])+1
        df_new_copy.loc[location,'基金类型']='短期纯债'
        
    if i in HHZJ_list:
        location = int(np.where(df_new_copy['基金代码']== i )[0])+1
        df_new_copy.loc[location,'基金类型']='混合债基'
        
    if i in KZZ_list:
        location = int(np.where(df_new_copy['基金代码']== i )[0])+1
        df_new_copy.loc[location,'基金类型']='可转债'
    
    if i in BDZSX_list:
        location = int(np.where(df_new_copy['基金代码']== i )[0])+1
        df_new_copy.loc[location,'基金类型']='被动指数型'
    
    if i in ZQZSX_list:
        location = int(np.where(df_new_copy['基金代码']== i )[0])+1
        df_new_copy.loc[location,'基金类型']='主动指数型'
        
    else:
        pass
    
df_new_copy
    

C:\Users\lujiani\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\lujiani\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\lujiani\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\lujiani\a

,基金代码,基金名称,基金类型,基金经理,今日净值,昨日净值,累计净值,今日涨跌幅,过去一月涨跌幅,过去三月涨跌幅,过去六月涨跌幅,过去一年涨跌幅,过去三年涨跌幅,成立以来涨跌幅
1,000001,华夏成长混合,混合型,董阳阳,1.2089,1.2050,3.6660,+0.33%,3.61%,0.75%,16.65%,18.90%,25.66%,591.02%
2,000003,中海可转债债券A,可转债,彭海平,0.7625,0.7660,0.9760,-0.46%,-4.01%,-8.70%,3.79%,13.48%,0.26%,-9.94%
3,000004,中海可转债债券C,可转债,彭海平,0.7615,0.7650,0.9750,-0.46%,-4.02%,-8.71%,3.52%,13.33%,-0.39%,-10.06%
4,000005,嘉实增强信用定期债券,长期纯债,刘宁,1.0405,1.0400,1.3820,+0.04%,-1.42%,0.69%,3.96%,7.94%,21.49%,44.58%
5,000006,西部利得量化成长混合,混合型,盛丰衍,1.4667,1.4656,1.4656,+0.08%,5.78%,3.95%,30.72%,52.86%,--,46.56%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9249,970002,天泽六个月定开债C,长期纯债,吴一纯,0.9998,0.9998,0.9998,0.00%,--,--,--,--,--,-0.02%
9250,970003,安信瑞鸿中短债A,长期纯债,杨坚丽,1.0233,1.0233,1.4747,0.00%,--,--,--,--,--,0.03%
9251,970004,安信瑞鸿中短债B,长期纯债,杨坚丽,1.0234,1.0234,1.0234,0.00%,--,--,--,--,--,0.03%
9252,970005,安信瑞鸿中短债C,长期纯债,杨坚丽,1.0233,1.0233,1.0233,0.00%,--,--,--,--,--,0.02%


In [73]:
df_new_copy.to_csv('C:\\Users\\lujiani\\Desktop\\东方证券\\珺容\\基金数据库\\20200604.csv',index=False, encoding='utf_8_sig') ###生成excel